# Proyecto de programación "*Deep Vision para tareas de clasificación*"


## Enunciado

En esta actividad, el alumno debe **evaluar y comparar estrategias** para la **clasificación de imágenes** empleando el **dataset asignado**. El alumno deberá resolver el reto proponiendo una solución válida **basada en aprendizaje profundo**, más concretamente en redes neuronales convolucionales (**CNNs**). Será indispensable que la solución propuesta siga el **pipeline visto en clase** para resolver este tipo de tareas de inteligencia artificial:

1.   **Carga** del conjunto de datos
2.   **Inspección** del conjunto de datos
3.   **Acondicionamiento** del conjunto de datos
4.   Desarrollo de la **arquitectura** de red neuronal y **entrenamiento** de la solución
5.   **Monitorización** del proceso de **entrenamiento** para la toma de decisiones
6.   **Evaluación** del modelo predictivo.

### Entrenar desde cero o *from scratch*

La primera estrategia a comparar será una **red neuronal profunda** que el **alumno debe diseñar, entrenar y optimizar**. Se debe **justificar empíricamente** las decisiones que llevaron a la selección de la **arquitectura e hiperparámetros final**. Se espera que el alumno utilice todas las **técnicas de regularización** mostradas en clase de forma justificada para la mejora del rendimiento de la red neuronal (*weight regularization*, *dropout*, *batch normalization*, *data augmentation*, etc.).


## Normas a seguir

- Será **posible** realizar el **trabajo por parejas**. 
- En caso de trabajar por parejas, se debe entregar un **ÚNICO FICHERO PDF POR ALUMNO** que incluya las instrucciones presentes en el Noteboook y su **EJECUCIÓN**. Debe aparecer todo el proceso llevado a cabo en cada estrategia (i.e. carga de datos, inspección de datos, acondicionamiento, proceso de entrenamiento y proceso de validación del modelo).
- **La memoria del trabajo** (el fichero PDF mencionado en el punto anterior) deberá **subirla cada integrante del grupo** (aunque se trate de un documento idéntico) a la tarea que se habilitará.
- Se recomienda trabajar respecto a un directorio base (**BASE_FOLDER**) para facilitar el trabajo en equipo. En este notebook se incluye un ejemplo de cómo almacenar/cargar datos utilizando un directorio base.
- Las **redes propuestas** deben estar **entrenadas** (y **EVIDENCIAR este proceso en el documento PDF**). La entrega de una **red sin entrenar** supondrá **perdida de puntos**.
- Si se desea **evidenciar alguna métrica** del proceso de entrenamiento (precisión, pérdida, etc.), estas deben ser generadas.
- Todos los **gráficos** que se deseen mostrar deberán **generarse en el Notebook** para que tras la conversión aparezcan en el documento PDF.

## *Tips* para realizar la actividad con éxito
- Los **datos** se podrán cargar directamente **desde** la plataforma **Kaggle** mediante su API (https://github.com/Kaggle/kaggle-api) o se podrán descargar en local. En este Notebook se incluye un ejemplo de como hacer la carga directa desde **Kaggle**. Se recomienda generar una función que aborde esta tarea.
- El **documento PDF a entregar** como solución de la actividad se debe **generar automáticamente desde el fichero ".ipynb"**. En este Notebook se incluye un ejemplo de como hacerlo.
- **Generar secciones y subsecciones en el Colab Notebook** supondrá que el documento **PDF generado** queda totalmente **ordenado** facilitando la evaluación al docente.
- Se recomienda encarecidamente **incluir comentarios aclaratorios** de todo el desarrollo y de las decisiones tomadas. 
- Es muy recomendable crear una **última sección** de texto en el Colab Notebook en la que se discutan los diferentes modelos obtenidos y se extraigan las **conclusiones** pertinentes.

## Criterios de evaluación

- **Seguimiento** de las **normas establecidas** en la actividad (detalladas anteriormente).
- Creación de una **solución que resuelva la tarea de clasificación**.
- **Claridad** en la creación de la solución, en las justificaciones sobre la toma de decisiones llevada a cabo así como en las comparativas y conclusiones finales.
- **Efectividad** al presentar las comparaciones entre métricas de evaluación de ambas estrategias.
- **Demostración** de la utilización de **técnicas de regularización** para mejorar el rendimiento de los modelos.

## Datasets disponibles:

- https://www.kaggle.com/c/histopathologic-cancer-detection/data
- https://www.kaggle.com/c/cassava-leaf-disease-classification/data
- https://www.kaggle.com/c/plant-seedlings-classification/data
- https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/data
- https://www.kaggle.com/c/dog-breed-identification/data

## Bloques de código de referencia

Los siguientes bloques de código son una referencia para poder partir de una estructura inicial. Podeis usarlos o generar otra estructura que se adecúe más a vuestro proyecto

### Descarga y ubicación de datos
Los datos descargados de kaggle los hemos guardado en una carpeta llamada data, dentro está la carpeta train que es la única que podemos usar 

In [ ]:
# Imports 
import shutil
import random
from pathlib import Path
from pprint import pprint
from collections import defaultdict

import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit


Dejamos los parámetros del notebook configurados aquí. Para probar distintos tipos de entrenamiento o redes, escribir aquí y ejecutar el resto de celdas 

In [ ]:
# Params 
# Train 
batch_size = 8
monitor = 'val_loss'
learning_rate = 1e-4
epochs = 50
early_stopping_patience = 4
train_backbone = True
version = 0
plateau_factor = 0.5
plateau_patience = 2

# Model 
input_shape = (224, 224, 3)
model_name = 'resnet50'

# Data 
train_data_path = 'data/train'
test_data_path = 'data/test'
original_data_path = 'data/original'
class_names = [str(p.name) for p in Path('data/original').glob('*')]


Veamos el número de elementos por clase

In [ ]:
paths_dataset = list(Path(original_data_path).rglob('*.png'))
dict_dataset = defaultdict(list)

for p in paths_dataset:
    dict_dataset[p.parent.name].append(str(p))

for k in dict_dataset.keys():
    print(f'La clase {k} tiene {len(dict_dataset[k])} elementos con proporción {len(dict_dataset[k])/len(paths_dataset)}')

plt.rcParams["figure.figsize"] = (8, 8)
plt.barh(y=list(dict_dataset.keys()), width=[len(dict_dataset[k]) for k in dict_dataset.keys()])
plt.title('Distribution over classes')
plt.show()


Mostremos algunos datos para ver qué tenemos entre manos

In [ ]:
sample_path = random.choice(paths_dataset)

plt.imshow(cv2.imread(str(sample_path))[..., ::-1])
plt.title(sample_path.parent.name)
plt.show()

Definimos diccionario con clave el nombre de la clase y valor los índices que tiene cada clase

In [ ]:
names_to_index = {k: i for i, k in enumerate(dict_dataset.keys())}
names_proportion = {k: len(dict_dataset[k])/len(paths_dataset) for k in dict_dataset.keys()}
print('Los índices por clase son: ')
pprint(names_to_index)
print('\n')
print('La distribución de los datos por clase es:')
pprint(names_proportion)

plt.pie(names_proportion.values(), labels=names_proportion.keys())
plt.title('Distribución de los datos en el dataset')
plt.show()

In [ ]:
X = np.array(paths_dataset)
Y = np.array([names_to_index[p.parent.name] for p in X])
assert len(X) == len(Y)

# Hacemos el split estratificado en train y test
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=42)

sss.get_n_splits(X, Y)
for train_index, test_index in sss.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
assert len(X_train) == len(Y_train)
assert len(X_test) == len(Y_test)

unique, counts = np.unique(Y_test, return_counts=True)
counts = counts / counts.sum()
print('La distibución por clase en los datos de test es:')
names_proportion_test = dict(zip(dict_dataset.keys(), counts))
pprint(names_proportion_test)

plt.pie(names_proportion_test.values(), labels=names_proportion_test.keys())
plt.title('Distribución por clase de los datos en el dataset de test')
plt.show()


Ahora hagamos una partición física de los datos, dejamos el código dentro de una función si se quiere ejecutar descomentar la línea comentada y correr la celda. Ojo!! Ejecutarla solo una vez

In [ ]:
def create_test_data(X_test: np.ndarray):
    for path in tqdm(X_test):
        path = Path(path)
        assert path.exists()
        dst = Path(str(path).replace('train', 'test'))
        dst.parent.mkdir(exist_ok=True, parents=True)
        if not dst.exists():
            shutil.move(path, dst)
        else:
            print(f'El archivo {dst} ya existe en test, lleva cuidado de no ejecutar muchas veces la función')
    return

# create_test_data(X_test=X_test)

In [ ]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    directory=train_data_path,
    labels='inferred', 
    label_mode='categorical', 
    class_names=class_names,
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=input_shape[:2], 
    shuffle=True, 
    seed=42, 
    validation_split=0.2, 
    subset='training', 
    interpolation='nearest', 
    follow_links=False, 
    crop_to_aspect_ratio=False,
)

val_dataset = keras.preprocessing.image_dataset_from_directory(
    directory=train_data_path,
    labels='inferred', 
    label_mode='categorical', 
    class_names=class_names,
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=input_shape[:2], 
    shuffle=True, 
    seed=42, 
    validation_split=0.2, 
    subset='validation', 
    interpolation='nearest', 
    follow_links=False, 
    crop_to_aspect_ratio=False,
)

train_dataset = train_dataset.prefetch(buffer_size=batch_size)
val_dataset = val_dataset.prefetch(buffer_size=batch_size)


In [ ]:
data_augmentation = keras.Sequential(
[
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation((-1, 1), fill_mode='reflect', interpolation='nearest'),
    keras.layers.experimental.preprocessing.RandomZoom(width_factor=(0, 0.2), height_factor=(0, 0.2), interpolation='nearest'),
]
)

Visualicemos algunos datos 

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(batch_size):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

Hagamos una función para poder seleccionar varios modelos y probar.

In [ ]:
def get_model(model_name: str, input_shape: tuple, train_backbone: bool):
    # TODO añadir mas modelos, resnet101, inception...
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    
    if model_name == 'mobilenetv2':
        x = keras.applications.mobilenet.preprocess_input(inputs)
        base_model = keras.applications.MobileNetV2(
            input_shape=input_shape,
            include_top=False,
            weights="imagenet",
        )

    elif model_name == 'resnet50':
        x = keras.applications.resnet50.preprocess_input(x)
        base_model = keras.applications.ResNet50V2(
            include_top=False,
            weights="imagenet",
            input_shape=input_shape,
        )
        
    base_model.trainable = train_backbone
    x = base_model(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(x.shape[-1] // 2, activation='relu', name='adria_layer_1')(x)
    x = keras.layers.Dropout(0.2)(x) 
    x = keras.layers.Dense(x.shape[-1] // 2, activation='relu', name='adria_layer_2')(x) 
    predictions = keras.layers.Dense(12, activation='softmax', name='predictions')(x) 
    model = keras.Model(inputs, predictions)

    return model

model = get_model(model_name, input_shape, train_backbone)
model.summary()


In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(f'weights/{model_name}/version_{version}', save_best_only=True, monitor=monitor),
    keras.callbacks.EarlyStopping(monitor=monitor, patience=early_stopping_patience, mode='auto'),
    keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=plateau_factor, patience=plateau_patience, mode='auto'),
    keras.callbacks.TensorBoard(log_dir=f'weights/{model_name}/version_{version}')
]


In [ ]:
# Optimizador Adam
# TODO custom crossentropy loss for weighted class version
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
    loss='categorical_crossentropy', 
    metrics=[keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.CategoricalAccuracy()])

# Entrenamos el modelo
H = model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=epochs, 
    verbose=1, 
    callbacks=callbacks
)


In [ ]:
# Gráficas losses
epochs_trained = len(H.history['loss'])
plt.style.use('ggplot')
plt.figure()
plt.plot(np.arange(0, epochs_trained), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, epochs_trained), H.history['val_loss'], label='val_loss')

plt.title(f'Training and Val Loss {model_name}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig(f'weights/{model_name}/version_{version}/losses.png')
plt.show()


In [ ]:
# Gráficas precision recall
plt.style.use('ggplot')
plt.figure()

plt.plot(np.arange(0, epochs_trained), H.history['precision'], label='train_precision')
plt.plot(np.arange(0, epochs_trained), H.history['recall'], label='train_recall')
plt.plot(np.arange(0, epochs_trained), H.history['categorical_accuracy'], label='train_categorical_accuracy')

plt.plot(np.arange(0, epochs_trained), H.history['val_precision'], label='val_precision')
plt.plot(np.arange(0, epochs_trained), H.history['val_recall'], label='val_recall')
plt.plot(np.arange(0, epochs_trained), H.history['val_categorical_accuracy'], label='val_categorical_accuracy')

plt.title(f'Training and val Accuracy/Precision/Recall {model_name}')
plt.xlabel('Epoch')
plt.ylabel('Precision/Recall')
plt.legend()
plt.savefig(f'weights/{model_name}/version_{version}/metrics.png')
plt.show()


In [ ]:
# TODO mejorar este código horrible  
del model
print('Loading model from checkpoint ...')
model = keras.models.load_model(f'weights/{model_name}/version_{version}')
print('Model loaded!')

test_dataset = keras.preprocessing.image_dataset_from_directory(
    directory=test_data_path,
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=input_shape[:2],
    shuffle=True,
    seed=42,
    validation_split=0.99999,
    subset='validation',
    interpolation='nearest',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

# Inferencia, preparada para inferir en forma de batch
preds = []
targets = []
for imgs, labels in tqdm(test_dataset):
    preds += [int(pred) for pred in tf.argmax(model.predict(imgs), axis=1)]
    targets += [int(label) for label in tf.argmax(labels, axis=1)]

# Métricas de clasificación 
print(classification_report(targets, preds, target_names=class_names))


## Ejemplo de generación de documento PDF a partir del Colab Notebook (fichero ".ipynb")

In [ ]:
# Ejecutando los siguientes comandos en la última celda del Colab Notebook se convierte de ".ipynb" a PDF
# En caso de querer ocultar la salida de una celda puesto que no tenga relevancia se debe insertar 
# el comando %%capture al inicio de la misma. Véase la celda que contiene !ls test en este Notebook.

In [ ]:
name_IPYNB_file = 'Proyecto_Programacion.ipynb'
get_ipython().system(
        "apt update >> /dev/null && apt install texlive-xetex texlive-fonts-recommended texlive-generic-recommended >> /dev/null"
    )
get_ipython().system(
            "jupyter nbconvert --output-dir='$BASE_FOLDER' '$BASE_FOLDER''$name_IPYNB_file' --to pdf"
        )